In [1]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

In [22]:
!pip install numpy==1.20.0 --user
!pip install numba==0.53 --user

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/LinkedInLearning/faster-pandas-2832038/main/Ch02/02_02/cart.csv')
df

,Customer,Item,Amount,Item Price
0,Rick,Wine,20,103.20
1,Morty,Almond Milk,1,10.04
2,Summer,Ice Cream,1,8.32
3,Beth,Comb,1,7.30
4,Jerry,Tequila,2,20.34


In [6]:
mask = df['Item Price'] > 10

In [8]:
df[mask]

,Customer,Item,Amount,Item Price
0,Rick,Wine,20,103.20
1,Morty,Almond Milk,1,10.04
4,Jerry,Tequila,2,20.34


In [96]:
conn = sqlite3.connect('logs.db', detect_types=sqlite3.PARSE_DECLTYPES)
sql1 = 'SELECT * FROM logs'

In [12]:
dff = pd.read_sql(sql1, conn)
dff

,time,origin,method,path,status_code,size
0,1995-08-01 00:00:01,in24.inetnebr.com,GET,/shuttle/missions/sts-68/news/sts-68-mcc-05.txt,200,1839
1,1995-08-01 00:00:07,uplherc.upl.com,GET,/,304,0
2,1995-08-01 00:00:08,uplherc.upl.com,GET,/images/ksclogo-medium.gif,304,0
3,1995-08-01 00:00:08,uplherc.upl.com,GET,/images/MOSAIC-logosmall.gif,304,0
4,1995-08-01 00:00:08,uplherc.upl.com,GET,/images/USA-logosmall.gif,304,0
...,...,...,...,...,...,...
9995,1995-08-01 07:54:40,ppp-14.flashnet.it,GET,/images/USA-logosmall.gif,200,234
9996,1995-08-01 07:54:41,204.238.216.51,GET,/images/ksclogo-medium.gif,304,0
9997,1995-08-01 07:54:42,204.238.216.51,GET,/images/USA-logosmall.gif,304,0
9998,1995-08-01 07:54:42,204.238.216.51,GET,/images/MOSAIC-logosmall.gif,304,0


In [16]:
%%timeit
tot = 0
for x,row in dff.iterrows():
    if row['status_code'] >=400:
        tot+= 1


2.77 s ± 459 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%timeit dff[dff['status_code'] >=400] #boolean indexing

1.27 ms ± 319 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Universal Functions

In [19]:
s =pd.Series(range(1000))
%timeit max(s)

283 µs ± 69.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [20]:
%timeit s.max() #pandas max function faster

108 µs ± 20.6 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## limitations of appending

In [2]:
def parse_time(ts):
    # [02/Jul/1995:16:30:08 -0400] for converting string time to datetime format
    time = datetime.strptime(ts, '[%d/%b/%Y:%H:%M:%S %z]')
    return time.replace(tzinfo=None)  # Remove time zone

def parse_line(line): #for parsing the line
    fields = line.split()
    size = 0 if fields[-1] == '-' else int(fields[-1])
    return {
        'origin': fields[0],
        'time': parse_time(fields[3] + ' ' + fields[4]),
        'method': fields[5][1:],  # Remove leading "
        'path': fields[6],
        'status_code': int(fields[-2]),
        'size': size,
    }

In [23]:
with open ("log.txt", "r") as fp:
    lines = fp.readlines()

In [27]:
%%timeit
df = pd.DataFrame()
for line in lines:
    df = df.append(parse_line(line), ignore_index= True)

1.82 s ± 14.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
# Dataframes are slow with append, python lists are very fast.
# from records to convert DataFrame to a NumPy record array
%timeit df = pd.DataFrame.from_records(parse_line(line) for line in lines) 

25.8 ms ± 707 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [31]:
df = pd.DataFrame.from_records(parse_line(line) for line in lines)
df

,origin,time,method,path,status_code,size
0,in24.inetnebr.com,1995-08-01 00:00:01,GET,/shuttle/missions/sts-68/news/sts-68-mcc-05.txt,200,1839
1,uplherc.upl.com,1995-08-01 00:00:07,GET,/,304,0
2,uplherc.upl.com,1995-08-01 00:00:08,GET,/images/ksclogo-medium.gif,304,0
3,uplherc.upl.com,1995-08-01 00:00:08,GET,/images/MOSAIC-logosmall.gif,304,0
4,uplherc.upl.com,1995-08-01 00:00:08,GET,/images/USA-logosmall.gif,304,0
...,...,...,...,...,...,...
995,www-d1.proxy.aol.com,1995-08-01 00:36:25,GET,/elv/vidpicp.htm,200,4251
996,pme609.onramp.awinc.com,1995-08-01 00:36:26,GET,/shuttle/resources/orbiters/discovery-logo.gif,200,4179
997,marimo.kushiro-ct.ac.jp,1995-08-01 00:36:27,GET,/shuttle/countdown/count70.gif,200,46573
998,in24.inetnebr.com,1995-08-01 00:36:28,GET,/shuttle/missions/sts-59/news/,200,6122


In [4]:
size = 50000
df = pd.DataFrame({
                'a': np.random.randint(1,1000, size),
                'b': np.random.randint(1,1000, size),
                'c': np.random.randint(1,1000, size),
})

In [5]:
%%timeit
total = 0
for _, row in df.iterrows():
    total+= row.max()

17.3 s ± 2.31 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit df.apply(np.max, axis =1, raw = True).sum() #raw supplies the row as numpy array to sum()

1.15 s ± 265 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
conn = sqlite3.connect('data3/logs.db', detect_types=sqlite3.PARSE_DECLTYPES)
sql1 = 'SELECT * FROM logs'

dff = pd.read_sql(sql1, conn)
dff

,time,origin,method,path,status_code,size
0,1995-08-01 00:00:01,in24.inetnebr.com,GET,/shuttle/missions/sts-68/news/sts-68-mcc-05.txt,200,1839
1,1995-08-01 00:00:07,uplherc.upl.com,GET,/,304,0
2,1995-08-01 00:00:08,uplherc.upl.com,GET,/images/ksclogo-medium.gif,304,0
3,1995-08-01 00:00:08,uplherc.upl.com,GET,/images/MOSAIC-logosmall.gif,304,0
4,1995-08-01 00:00:08,uplherc.upl.com,GET,/images/USA-logosmall.gif,304,0
...,...,...,...,...,...,...
9995,1995-08-01 07:54:40,ppp-14.flashnet.it,GET,/images/USA-logosmall.gif,200,234
9996,1995-08-01 07:54:41,204.238.216.51,GET,/images/ksclogo-medium.gif,304,0
9997,1995-08-01 07:54:42,204.238.216.51,GET,/images/USA-logosmall.gif,304,0
9998,1995-08-01 07:54:42,204.238.216.51,GET,/images/MOSAIC-logosmall.gif,304,0


In [9]:
with open('data3/ips.txt', 'r') as fp:
    ips = [line.strip() for line in fp]

In [18]:
%timeit dff['origin'].isin(ips)

41.6 ms ± 7.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
ips_set = set(ips)
%timeit dff['origin'].isin(ips_set)

101 ms ± 25.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
%timeit dff['origin'].apply(lambda v: v in ips_set)

6.25 ms ± 1.76 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
is_bad = ips_set.__contains__
%timeit dff['origin'].apply(is_bad)

3.48 ms ± 1.04 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [31]:
df = pd.read_csv('logs.csv', parse_dates= ['time'])
df

,origin,time,method,path,status_code,size
0,uplherc.upl.com,1995-08-01 00:00:10,GET,/images/WORLD-logosmall.gif,304,0
1,133.43.96.45,1995-08-01 00:00:25,GET,/history/apollo/images/apollo-logo1.gif,200,1173
2,133.68.18.180,1995-08-01 00:01:13,GET,/persons/nasa-cm/jmd-sm.gif,200,3660
3,www-d3.proxy.aol.com,1995-08-01 00:01:28,GET,/images/NASA-logosmall.gif,200,786
4,haraway.ucet.ufl.edu,1995-08-01 00:04:47,GET,"/cgi-bin/imagemap/countdown70?199,165",302,97
...,...,...,...,...,...,...
49995,alpc6.mpimf-heidelberg.mpg.de,1995-08-31 23:53:54,GET,/htbin/cdt_main.pl,200,3873
49996,cindy.yamato.ibm.co.jp,1995-08-31 23:54:40,GET,/images/kscmap-tiny.gif,200,2537
49997,cys-cap-9.wyoming.com,1995-08-31 23:55:01,GET,/shuttle/missions/sts-71/movies/movies.html,200,3381
49998,203.243.250.7,1995-08-31 23:55:10,GET,/facilities/vab.html,200,4045


In [29]:
df['time'][32]

def is_morning(ts):
    t = pd.to_datetime(ts)
    return t.hour >=6 and t.hour <12 #to return the morning time from log files

In [30]:
%timeit df[df['time'].apply(is_morning)] #not a good choice and is slow as each time str is passed when function calls so use
#parse_dates in pd.read_csv

14.3 s ± 1.13 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
%timeit df[(df['time'].dt.hour >=6) & (df['time'].dt.hour <12)]

38.2 ms ± 5.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Find how many rides in 2016 were in the afternoon of weekend or holiday.
- Afternoon: Between noon to 6pm
- Weekend: Saturday or Sunday
- Holiday: See holidays_2016 below

In [85]:
# 2016 public holidays

from calendar import SATURDAY, SUNDAY
holidays_2016 = [
    '2016-01-01',  # new year
    '2016-01-18',  # MLK
    '2016-05-30',  # memorial
    '2016-07-04',  # independence
    '2016-09-05',  # labor
    '2016-11-11',  # veterans
    '2016-11-24',  # thanksgiving
    '2016-12-26',  # christmas
]


def load_df(file_name):
    """Load data from CSV to DataFrame"""
    return pd.read_csv(
        file_name,
        parse_dates={'time': ['Checkout Date', 'Checkout Time']},
    )


def vacation_rides(df):
    """Return only rows that are in holiday afternoon"""
    mask_2016 = df['time'].dt.year == 2016
    
    holiday_mask = (
        (df['time'].dt.floor('d').isin(holidays_2016)) |
        (df['time'].dt.weekday.isin([SATURDAY, SUNDAY]))
    )

    afternoon_mask = (df['time'].dt.hour >=12) & (df['time'].dt.hour <6)

    return df[mask_2016 & holiday_mask & afternoon_mask]

In [66]:
"""df = pd.read_csv('data3/austin-bikes.csv')
df['time'] = pd.to_datetime(df['Checkout Date'] + ' ' + df['Checkout Time'])
df.drop(['Checkout Date', 'Checkout Time'], axis = 1, inplace = True)
df.head(2)"""

"df = pd.read_csv('data3/austin-bikes.csv')\ndf['time'] = pd.to_datetime(df['Checkout Date'] + ' ' + df['Checkout Time'])\ndf.drop(['Checkout Date', 'Checkout Time'], axis = 1, inplace = True)\ndf.head(2)"

In [87]:
df = load_df('data3/austin-bikes.csv')

In [88]:
%timeit vacation_rides(df)

24 ms ± 2.81 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Pandas Performance

In [90]:
df = pd.read_csv('data3/austin-bikes.csv', low_memory=False)
df.head(2)

,Trip ID,Membership Type,Bicycle ID,Checkout Date,Checkout Time,Checkout Kiosk ID,Checkout Kiosk,Return Kiosk ID,Return Kiosk,Trip Duration Minutes,Month,Year
0,9900334132,24-Hour Kiosk (Austin B-cycle),839,12/22/2013,13:12:00,2495.0,4th & Congress,2495,4th & Congress,53,12.0,2013.0
1,9900334089,24-Hour Kiosk (Austin B-cycle),283,12/22/2013,10:12:00,2501.0,5th & Bowie,2495,4th & Congress,9,12.0,2013.0


In [93]:
%timeit df['Trip Duration Minutes'].max()

48 µs ± 595 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [95]:
%timeit df['Trip Duration Minutes'].values.max()

9.53 µs ± 68.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [94]:
%timeit max(df['Trip Duration Minutes'])

964 µs ± 9.81 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [102]:
df = pd.read_sql(sql1, conn)
df.head(3)

,time,origin,method,path,status_code,size
0,1995-08-01 00:00:01,in24.inetnebr.com,GET,/shuttle/missions/sts-68/news/sts-68-mcc-05.txt,200,1839
1,1995-08-01 00:00:07,uplherc.upl.com,GET,/,304,0
2,1995-08-01 00:00:08,uplherc.upl.com,GET,/images/ksclogo-medium.gif,304,0


In [103]:
df = pd.concat([df]*1000)

In [104]:
%timeit df[(df['method'] == 'GET') & df['status_code'] >=400]

701 ms ± 28.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [106]:
%timeit df.query('method == "GET" & status_code >= 400') #more efficient for large data and complex queries

462 ms ± 26.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [108]:
df = pd.read_csv('data3/taxi.csv')
df.head(4)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2018-05-01 00:13:56,2018-05-01 00:22:46,1,1.6,1,N,230,50,1,8.0,0.5,0.5,1.85,0.0,0.3,11.15
1,1,2018-05-01 00:23:26,2018-05-01 00:29:56,1,1.7,1,N,263,239,1,7.5,0.5,0.5,2.00,0.0,0.3,10.80
2,1,2018-05-01 00:36:23,2018-05-01 00:48:26,2,2.6,1,N,239,152,1,12.0,0.5,0.5,1.00,0.0,0.3,14.30
3,1,2018-05-01 00:26:12,2018-05-01 00:27:05,1,0.0,1,N,145,145,1,2.5,0.5,0.5,9.63,0.0,0.3,13.43


In [109]:
ndf = pd.DataFrame([
                    (1, 'Creative'),
                    (2, 'VeriFone'),
                    (4, 'BigApple'),], columns=['id', 'Vendor'])
ndf

,id,Vendor
0,1,Creative
1,2,VeriFone
2,4,BigApple


In [111]:
%timeit pd.merge(df, ndf, left_on='VendorID', right_on = 'id', how = 'left')

130 ms ± 4.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [112]:
ndfi = ndf.set_index('id')
%timeit pd.merge(df, ndfi, left_on='VendorID', right_index = True, how = 'left')

74.1 ms ± 3.88 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


"""What is the median trip duration in 2017, only in active kiosks?
- Trip data in austin-bikes.csv.xz
- Kiosk status data in austin-kiosk.csv
"""

In [132]:
bike_df = pd.read_csv('data3/austin-bikes (2).csv', low_memory = False)
kiosk_df = pd.read_csv('https://raw.githubusercontent.com/LinkedInLearning/faster-pandas-2832038/main/Ch04/challenge/austin-kiosk.csv',
index_col = 'Kiosk ID')

In [138]:
df = pd.merge(bike_df, kiosk_df, left_on = 'Checkout Kiosk ID', right_index = True)
active_2017 = df.query('`Kiosk Status` == "active" & `Year` ==  2017 & `Trip Duration Minutes` > 0')
active_2017['Trip Duration Minutes'].median()

16.0

## Saving Memory 

In [42]:
df = pd.read_csv('data3/taxi.csv')
df.head(3)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2018-05-01 00:13:56,2018-05-01 00:22:46,1,1.6,1,N,230,50,1,8.0,0.5,0.5,1.85,0.0,0.3,11.15
1,1,2018-05-01 00:23:26,2018-05-01 00:29:56,1,1.7,1,N,263,239,1,7.5,0.5,0.5,2.00,0.0,0.3,10.80
2,1,2018-05-01 00:36:23,2018-05-01 00:48:26,2,2.6,1,N,239,152,1,12.0,0.5,0.5,1.00,0.0,0.3,14.30


In [43]:
df.shape

(499999, 17)

In [19]:
df.dtypes

VendorID                   int64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count            int64
trip_distance            float64
RatecodeID                 int64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type               int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
dtype: object

In [22]:
mb = 2**20
df.memory_usage(deep = True)/mb, df.memory_usage(deep = True).sum()/mb

(Index                     0.000122
 VendorID                  3.814690
 tpep_pickup_datetime     36.239552
 tpep_dropoff_datetime    36.239552
 passenger_count           3.814690
 trip_distance             3.814690
 RatecodeID                3.814690
 store_and_fwd_flag       27.656500
 PULocationID              3.814690
 DOLocationID              3.814690
 payment_type              3.814690
 fare_amount               3.814690
 extra                     3.814690
 mta_tax                   3.814690
 tip_amount                3.814690
 tolls_amount              3.814690
 improvement_surcharge     3.814690
 total_amount              3.814690
 dtype: float64,
 153.54137992858887)

In [23]:
ta32 = df['fare_amount'].astype('float32')
ta32.memory_usage(deep =True)/mb

1.9074668884277344

## Loading parts of Data

In [44]:
df = pd.read_csv('data3/taxi.csv', usecols= ['VendorID', 'total_amount'], chunksize = 100000)
df # now it is not a dataframe but a textfile reader which is an iterator

In [45]:
for df in pd.read_csv('data3/taxi.csv', usecols= ['VendorID', 'total_amount'], chunksize = 100000):
    print(len(df))

100000
100000
100000
100000
99999


In [47]:
revs = [] #calculating the revenue in parts

for df in pd.read_csv('data3/taxi.csv', usecols= ['VendorID', 'total_amount'], chunksize = 100000):
    rev = df.groupby('VendorID').sum()
    revs.append(rev)
revs

[          total_amount
 VendorID              
 1            674987.76
 2            914002.85,
           total_amount
 VendorID              
 1            754956.37
 2            970064.08,
           total_amount
 VendorID              
 1            674117.09
 2            961032.99,
           total_amount
 VendorID              
 1            676086.16
 2            919557.04,
           total_amount
 VendorID              
 1            768881.73
 2            989439.62
 4                64.86]

In [48]:
pd.concat(revs).groupby(level = 0).sum() #to concate all revenues(level = 0 to unstack it)

,total_amount
VendorID,
1,3549029.11
2,4754096.58
4,64.86


""" Integers are more memory efficient and comparisions are faster than strings and"""

In [50]:
df['VendorID'].head(10)

400000    2
400001    1
400002    1
400003    2
400004    2
400005    2
400006    2
400007    2
400008    2
400009    1
Name: VendorID, dtype: int64

In [52]:
vendors = df['VendorID'].map({1: 'Creative', 2: 'VeriFone', 3: 'BigApple'})
vendors.head(3)

400000    VeriFone
400001    Creative
400002    Creative
Name: VendorID, dtype: object

In [55]:
cat_vendors = vendors.astype('category')
cat_vendors.sample(3)

472395    Creative
462586    Creative
461971    VeriFone
Name: VendorID, dtype: category
Categories (2, object): ['Creative', 'VeriFone']

In [56]:
vendors.memory_usage(deep = True)/ cat_vendors.memory_usage(deep = True)

64.7587402484831

"""Calculate the minimal and maximal distance driven from the data at
    taxi.csv.xz
Consume as little memory as possible and don't load more than 50,000 rows at a
time.
"""

In [61]:
min_dist, max_dist = float('inf'), float('-inf')
dfs = pd.read_csv('data3/taxi.csv', usecols= ['trip_distance'], chunksize = 50000)
for df in dfs:
    desc = df['trip_distance'].describe()
    min_dist = min(min_dist, desc['min'])
    max_dist = max(max_dist, desc['max'])
print(f'minimum and maximum distance {min_dist}, {max_dist}')

minimum and maximum distance 0.0, 111.1


## Fast Serialization

In [104]:
import sqlite3
conn = sqlite3.connect('data3/bikes.db')
query = " select name,sql from sqlite_master where type = 'table' " #metadata table

In [81]:
for name, sql in conn.execute(query):
    print(name) #name of table
    print(sql)

bike_rides
CREATE TABLE bike_rides(
  year INT,
  month INT,
  day INT,
  trip_id INT,
  bike_id INT,
  duration INT
)


In [82]:
conn.execute('SELECT MAX(YEAR), MIN(year) FROM bike_rides').fetchall()

[(2017, 2016)]

In [84]:
query2 = 'SELECT year, month, duration FROM bike_rides WHERE year== 2016 or 2017'
dfq = pd.read_sql(query2, conn)
dfq.head(5)

,year,month,duration
0,2017,5,38
1,2017,5,21
2,2017,1,55
3,2017,5,25
4,2017,1,47


In [87]:
dfq.groupby(['year', 'month'])['duration'].median()

year  month
2016  1        14.0
      2        15.0
      3        14.0
      5        15.0
      6        14.0
      7        15.0
      8        13.0
      9        15.0
      10       16.0
      11       14.0
2017  1        14.0
      2        16.0
      3        14.0
      4        18.0
      5        18.0
      6        16.0
      7        16.0
      8        13.0
      9        17.0
      10       16.0
      11       18.0
      12       16.0
Name: duration, dtype: float64

### Optimizing with HDF5

In [90]:
store = pd.HDFStore('data3/stocks.h5')
store.keys() #It contains single table with name stocks

['/stocks']

In [100]:
df = store['stocks']
df.head(5)

,symbol,open,high,low,close,adj close,volume
date,,,,,,,
1962-01-02,IBM,7.713333,7.713333,7.626667,7.626667,0.609973,387200.0
1962-01-03,IBM,7.626667,7.693333,7.626667,7.693333,0.615304,288000.0
1962-01-04,IBM,7.693333,7.693333,7.613333,7.616667,0.609173,256000.0
1962-01-05,IBM,7.606667,7.606667,7.453333,7.466667,0.597176,363200.0
1962-01-08,IBM,7.460000,7.460000,7.266667,7.326667,0.585979,544000.0


In [97]:
store.select('stocks', stop = 4) #to look some of the data before loading all to memory

,symbol,open,high,low,close,adj close,volume
date,,,,,,,
1962-01-02,IBM,7.713333,7.713333,7.626667,7.626667,0.609973,387200.0
1962-01-03,IBM,7.626667,7.693333,7.626667,7.693333,0.615304,288000.0
1962-01-04,IBM,7.693333,7.693333,7.613333,7.616667,0.609173,256000.0
1962-01-05,IBM,7.606667,7.606667,7.453333,7.466667,0.597176,363200.0


In [98]:
s = store.get_storer('stocks') 
s.nrows

88839

In [103]:
df = store.select('stocks', '(index >= "2010") & (index < "2012")', columns = ['symbol', 'close', 'volume'])
df.index.min(), df.index.max(), df.columns

(Timestamp('2010-01-04 00:00:00'),
 Timestamp('2011-12-30 00:00:00'),
 Index(['symbol', 'close', 'volume'], dtype='object'))

"""Using "bikes.db", find the 5 bikes (using "bike_id") that has the biggest
90% quantile of ride duration in the first quarter of 2017.
"""

In [108]:
query3 = " select bike_id, duration from bike_rides WHERE year==2017 AND month < 4 "
dfsql = pd.read_sql(query3, conn)
out = dfsql.groupby('bike_id')['duration'].quantile(0.9)
print(out.sort_values(ascending = False)[:5])

bike_id
393    124.6
68     111.2
832    100.6
326     96.6
429     96.0
Name: duration, dtype: float64


## Numba and Cython

In [2]:
def add(a,b):
    return a+b

In [9]:
import dis
dis.dis(add)

  2           0 LOAD_FAST                0 (a)
              2 LOAD_FAST                1 (b)
              4 BINARY_ADD
              6 RETURN_VALUE


In [ ]:
import numba
@numba.jit
def jit_add(a,b):
    return a+b

In [ ]:
type(jit_add) #returns a numba CPU Dispatcher 
jit_add.overloads() # returns the dictionary between types and compiled code
jit_add(1,2)
jit_add.overloads.keys()
fn = jit_add.get_overload((numba.int64, numba.int64)) #returns a compiled code
fn(2,3)
jit_add(1.0, 2.0)
jit_add.overloads.keys()

In [26]:
s = pd.Series(np.random.randint( -3, 200, 1000000))
def relu(num):
    if num < 0:
        return 0
    return num

In [28]:
%timeit s.apply(relu) #slower than numba

302 ms ± 6.11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
@numba.vectorize
def vect_relu(num):
    if num <0:
        return 0
    return num

In [ ]:
%timeit vect_relu(s) #faster

In [ ]:
#Python code in relu1.py

def relu(num):
    if num <0:
        return 0
    return num

In [ ]:
#cython: language_level=3 in relu.pyx
def relu(double num):
    if num <0:
        return 0
    return num

In [ ]:
#cython magic 

%load_ext cython

%%cython
def cy_relu(double n):
    if n < 0:
        return 0
    return n

In [ ]:
%timeit cy_relu(-3)

## Alternative DataFrames

In [ ]:
#Dask
#Vaex
#Modin
#Rapids